In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch
import pickle
import pandas as pd
import gradio as gr

bi_encoder = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
corpus_embeddings=pd.read_pickle("corpus_embeddings.pkl")
corpus=pd.read_pickle("corpus.pkl")

In [44]:
def search(query,top_k=100):
#     print("Top 5 Answer by the NSE:")
#     print()
    ans=[]
    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, corpus[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    
    for idx, hit in enumerate(hits[0:5]):
        ans.append(corpus[hit['corpus_id']])
    return ans[0],ans[1],ans[2],ans[3],ans[4]

exp=["Who is steve jobs","What is coldplay","What is a turing test?","What is the most interesting thing about our universe?","What are the most beautiful places on earth?"]

desc="This is a semantic search engine powered by SentenceTransformers (Nils_Reimers) with a retrieval and reranking system on Wikipedia corous. This will return the top 5 results. Search a query and see how good model is doing."

inp=gr.inputs.Textbox(lines=1, placeholder=None, default="", label="search you query here")
out=gr.outputs.Textbox(type="auto",label="search results")

iface = gr.Interface(fn=search, inputs=inp, outputs=[out,out,out,out,out],examples=exp,article=desc,title="Neural Search Engine",theme="huggingface",layout='vertical')
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7886/
Running on public URL: https://24291.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7886/',
 'https://24291.gradio.app')

In [13]:
search("where is france")

('France ( or ; ), officially the French Republic (, ), is a country whose metropolitan territory is located in Western Europe and that also comprises various overseas islands and territories located in other continents. Metropolitan France extends from the Mediterranean Sea to the English Channel and the North Sea, and from the Rhine to the Atlantic Ocean. It is often referred to as "L’Hexagone" ("The Hexagon") because of the shape of its territory. France is a unitary semi-presidential republic with its main ideals expressed in the Declaration of the Rights of Man and of the Citizen.',
 'The capital of France is Paris. In the course of history, the national capital has been in many locations other than Paris.',
 'Metropolitan France ( or "la Métropole") is the part of France that is in Europe. It can also be described as mainland France plus the island of Corsica. By contrast, Overseas France ("France d\'outre-mer") is the collective name for all of the French overseas departments, t

In [ ]:
iface = gr.Interface(fn=search, inputs=["text"], outputs=["textbox","textbox","textbox","textbox","textbox"]).launch(share=True)